## Chapter 12. ntroduction to Modeling Libraries in Python
<a id='index'></a>

## Table of Content
- [13.1 Interfacing Between pandas and Model Code](#131)
- [13.2 Creating Model Descriptions with Patsy](#132)
    - [13.2.1 Data Transformations in Patsy Formulas](#1321)
    - [13.2.2 Categorical Data and Patsy](#1322)
- [13.3 Introduction to statsmodels](#133)
    - [13.3.1 Estimating Linear Models](#1331)
    - [13.3.2 Estimating Time Series Processes](#1332)

<hr>

In [2]:
import numpy as np
import pandas as pd

<hr>

## 13.1 Interfacing Between pandas and Model Code
<a id='131'></a>
An important part of the model development process is called ***feature engineering*** in machine learning. This can describe any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context. The data ***aggregation and GroupBy*** tools we have explored in this book are used often in a feature engineering context.

In [3]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
df2 = pd.DataFrame(data.values, columns = ['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In Chapter 12 we looked at pandas’s Categorical type and the pandas.get_dummies function. Suppose we had a non-numeric column in our example dataset:

In [8]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])

data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


If we wanted to replace the 'category' column with dummy variables, we create dummy variables, drop the 'category' column, and then join the result:

In [9]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


<hr>

## 13.2 Creating Model Descriptions with Patsy
<a id='132'></a>
Patsy is a Python library for describing statistical models (especially linear models) with a small string-based “formula syntax,” which is inspired by (but not exactly the same as) the formula syntax used by the R and S statistical programming languages.
Patsy is well supported for specifying linear models in statsmodels, so I will focus on some of the main features to help you get up and running. Patsy’s formulas are a special string syntax that looks like:

In [10]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
# The patsy.dmatrices function takes a formula string along with a dataset 
# (which can be a DataFrame or a dict of arrays) and produces design matrices for a linear model:
import patsy

In [12]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [13]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [14]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [15]:
# These Patsy DesignMatrix instances are NumPy ndarrays with additional metadata:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [16]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [17]:
# You can suppress the intercept by adding the term + 0 to the model
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [18]:
# The Patsy objects can be passed directly into algorithms like numpy.linalg.lstsq, 
# which performs an ordinary least squares regression:
coef, resid, _, _ = np.linalg.lstsq(X, y)
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [19]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### 13.2.1 Data Transformations in Patsy Formulas
<a id='1321'></a>

In [20]:
# You can mix Python code into Patsy formula
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [21]:
# Some commonly used variable transformations include standardizing (to mean 0 and variance 1) and 
# centering (subtracting the mean).
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [22]:
# The patsy.build_design_matrices function can apply transformations to new out-of-sample data using the saved 
# information from the original in-sample dataset:
new_data = pd.DataFrame({'x0': [6, 7, 8, 9], 
                         'x1': [3.1, -0.5, 0., 2.3],
                         'y': [1, 2, 3, 4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [23]:
# Because the plus symbol (+) in the context of Patsy formulas does not mean addition, 
# when you want to add columns from a dataset by name, you must wrap them in the special I function:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### 13.2.2 Categorical Data and Patsy
<a id='1322'></a>
When you use non-numeric terms in a Patsy formula, they are converted to dummy variables by default. If there is an intercept, one of the levels will be left out to avoid collinearity:

In [33]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
                     'key2': [0, 1, 0, 1, 0, 1, 0, 0],
                     'v1': [1, 2, 3, 4, 5, 6, 7, 8],
                     'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})

y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

If you omit the intercept from the model, then columns for each category value will be included in the model design matrix:

In [25]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [26]:
# Numeric columns can be interpreted as categorical with the C function:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

When you’re using multiple categorical terms in a model, things can be more compli‐ cated, as you can include interaction terms of the form ***key1:key2***, which can be used, for example, in ***analysis of variance (ANOVA)*** models:

In [36]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [37]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

<hr>

## 13.3 Introduction to statsmodels
<a id='133'></a>
Some kinds of models found in statsmodels include:
* Linear models, generalized linear models, and robust linear models
* Linear mixed effects models
* Analysis of variance (ANOVA) methods
* Time series processes and state space models
* Generalized method of moments

In [32]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 13.3.1 Estimating Linear Models
<a id='1331'></a>
Linear models in statsmodels have two different main interfaces: ***array-based*** and ***formula-based***.

Here, I wrote down the “***true***” model with known parameters ***beta***. In this case, ***dnorm*** is a helper function for generating normally distributed data with a particular mean and variance. 

In [35]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * np.random.randn(size)

# for reproducibility
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]

eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [36]:
X_model = sm.add_constant(X)
X_model

array([[  1.00000000e+00,  -1.29468492e-01,  -1.21275292e+00,
          5.04224878e-01],
       [  1.00000000e+00,   3.02910364e-01,  -4.35741756e-01,
         -2.54179861e-01],
       [  1.00000000e+00,  -3.28521889e-01,  -2.53015334e-02,
          1.38350968e-01],
       [  1.00000000e+00,  -3.51474705e-01,  -7.19605110e-01,
         -2.58214633e-01],
       [  1.00000000e+00,   1.24326880e+00,  -3.73799164e-01,
         -5.22629046e-01],
       [  1.00000000e+00,   8.81267227e-01,  -2.80898544e-02,
         -3.68960148e-01],
       [  1.00000000e+00,   5.87601006e-02,   8.48485492e-01,
         -1.18261588e+00],
       [  1.00000000e+00,   1.78191913e-01,   7.59823931e-01,
         -6.84173312e-02],
       [  1.00000000e+00,   4.86372577e-01,  -4.56615198e-01,
         -3.36269295e-01],
       [  1.00000000e+00,   7.88314544e-01,   1.22517962e+00,
         -5.93046604e-02],
       [  1.00000000e+00,   6.37002481e-01,  -4.09556235e-01,
          6.51724241e-01],
       [  1.00000000e

In [37]:
model = sm.OLS(y, X)
results = model.fit()

results.params

array([ 0.17826108,  0.22303962,  0.50095093])

In [39]:
# The summary method on results can print a model detailing diagnostic output of the model:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Mon, 08 Jan 2018   Prob (F-statistic):           7.44e-12
Time:                        23:52:36   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.001       0.073       0.283
x2             0.2230      0.046      4.818      0.000       0.131       0.315
x3             0.5010      0.080      6.237      0.000       0.342       0.660
==============================================================================
Omnibus:                        4.662   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                4.098
Skew:                           0.481   Prob(JB):                        0.129
Kurtosis:                       3.243   Cond. No.                         1.74
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# Suppose instead that all of the model parameters are in a DataFrame:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y

data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [43]:
# We also do not need to use add_constant when using formulas and pandas objects.

results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [44]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [45]:
# Given new out-of-sample data, you can compute predicted values given the estimated model parameters:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### 13.3.2 Estimating Time Series Processes
<a id='1332'></a>
Another class of models in statsmodels are for ***time series analysis***. Among these are autoregressive processes, Kalman filtering and other state space models, and multivariate autoregressive models.

In [47]:
import random

init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)

for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)
    
# This data has an AR(2) structure (two lags) with parameters 0.8 and –0.4. 
# When you fit an AR model, you may not know the number of lagged terms to include, 
# so you can fit the model with some larger number of lags:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

results.params

array([ 0.00791554,  0.71074082, -0.31689896, -0.06724519,  0.0041215 ,
       -0.00079061])

<hr>

[Back to top](#index)